# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

In [122]:
import pandas as pd
import numpy as np

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [123]:
data = pd.read_csv('Advertising.csv')
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [124]:
data = data.drop('Unnamed: 0', axis=1)

**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [125]:
data.isnull().sum().sort_values(ascending=False)

TV           0
radio        0
newspaper    0
sales        0
dtype: int64

*Пропусков нет*

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [126]:
aim_f = 'sales'
factors = list(data.drop(aim_f, axis=1).columns)
X = np.array(data[factors])
y = np.array(data[aim_f])
y = y.reshape((1,len(y))).T
y.shape

(200, 1)

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [127]:
X = np.column_stack((np.ones(X.shape[0]), X))
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [128]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [129]:
# Если честно, я бы писал также, через матричные операции , так что не виду смысла изобретать велосипед. 
# Тем более, что не вижу пути сделать код короче.

n_iter = 1000
m = X.shape[0]
n = X.shape[1]
w = np.zeros((n,1))

for i in range(n_iter):
    for k in range(n):
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        w[k] = (X[:,k] @ (y-h))
        
w


array([[ 41.56217205],
       [110.13144155],
       [ 73.52860638],
       [ -0.55006384]])

*А теперь объясню сам себе почему это работает*

*$\hat{y} = \sum_{i=0}^{m}{x_i * w_i } = h_k + x_k * w_k$ ,где $x_j$ - вектор столбцы признаков, $h_k$ - это вектор посчитанный без k-го признака.*
*Мы хотим посчитать число $w_k$. Домножим скалярно все на x_k и учитывая, что мы нормировали признаки в самом начале скалярный квадрат* *$x_k$ равен $1$.*

Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [130]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


Если вы все сделали верно, они должны практически совпасть!

*Совпадение до 9го знака есть :)*

In [131]:
#Проверим еще способность MSE, чтобы потом сравнить с стахостическим градиентным спуcком
#Я себе тут позволил использовать библиотечную функцию т.к. эта строка вне задания
from sklearn.metrics import mean_squared_error
mean_squared_error(model.predict(X),y)

2.784126314510936

## Стохастический градиентный спуск (6 баллов)

*Функции в этом разделе написаны с некоторой долей неаккуратности. В том смысле, что вектора в них надо подавать правильных размерностей (столбцы/строки подбирать). Я не стал заморачиваться с проверками там всякими, если честно, интереснее было реализовать алгоритм руками* 

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [132]:
X = np.array(data[factors]) #перезагрузим X в исходный вид
X = X - X.mean(axis=0)
X = X / np.linalg.norm(X, axis=0)


**Добавим единичный столбец**

In [133]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [134]:
def mse_error(x,y):
    return ((x-y)**2).sum()/len(x)

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [135]:
y_pred = np.ones(len(y))*(y.mean())
mse_error(y_pred,y.T)

27.085743750000002

**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [136]:
def lin_pred(X,w):
    return X @ w
    

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [137]:
def stoch_grad_step(X,y,w,train_ind,eta):
    #Т.к. это функуия для внутреннего пользования, я не заморачиваюсь с соответсвтием типов переменных
    #return X@w-y
    return (-((2*eta)/len(y)) * X[:,train_ind] @ (X @ w - y))[0]

*тут ниже небольшая проверка, что функция считает именно то, что в формуле указано. Проверял т.к. как раз запутался с векторами столбцами и строками и не сразу все заработало*

In [138]:
A = np.array([[1,1,1],[1,2,3],[1,3,4],[1,4,5]])
www = np.ones(3).reshape((-1,1))
b = np.ones(4).reshape((-1,1))
b


array([[1.],
       [1.],
       [1.],
       [1.]])

In [139]:
stoch_grad_step(X=A,y=b,w=www,train_ind=0,eta=1)

-11.5

*Тут проверка закончена. Функция stoch_grad_step работает корректно, если y и w - векторы столбцы т.е. массивы т на 1*

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [140]:
#После реализации и экспериметов с алгоритмом, появились наблюдения,
#которые попрошу прокомментировать проверяющего ментора (после блока с этим кодом markdown)
def stochastic_gradient_descent(X,y,w0,tempo,max_iter,min_dist):
    dist = 1000000
    errors = []
    it = 0
    w_now = w0.reshape(-1,1)
    while dist > min_dist:
        if it > max_iter:
            print('maximum iterations')
            break
        i = np.random.randint(0,len(w0))
        errors.append(mse_error(X@w_now,y))
        step = stoch_grad_step(X=X,y=y,w=w_now,train_ind=i,eta=tempo)
        #dist = step**2 # это как раз расстояние между старыми и новыми w
        w_now[i][0] += step
        it+=1
    
    return errors, w_now

***Вопросы и замечания***
1. *Я понимаю, что по сути убрал из кода проверку на расстояние между соседними w на каждой итерации, но я руководствовался некоторыми причинами, которые описал в следующих пунтах*
2. *Мы идем в этом методе вдоль одной координаты только на каждой итерации. Представим, что именно вдоль этой координаты движение очень маленькое, а направление спуска высокое по другим координатам. Тогда разница между w_n и w_(n-1) может быть очень маленькой в какой-то момент. И если смотреть критерий остановки в том, что значения близки, то алгоритм останавливается раньше времни, не достигнув настоящего минимума.*
3. *Именно это и получалось тут в результате экпериментов. Если оставить критерий с расстояниями, алгоритм останавливался с итоговой mse примерно равной mse наивного прогноза и в районе 30-60 итерации. Именно поэтому я убрал критерий малого расстояния между w на предыдущей и текущей итерациях (и из-за этого код несколько "поглупел" т.к. модно цикл поменять на while True).*

***ПРОСЬБА ПРОВЕРЯЮЩЕГО МЕНТОРА ПРОКОММЕНТИРОВАТЬ ЭТИ РАССУЖДЕНИЯ. ВЕРНЫ ЛИ ОНИ?***

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

In [141]:
err, w_res = stochastic_gradient_descent(X=X,y=y,w0=np.zeros(4),tempo=0.3,max_iter=10000,min_dist=0.0000001)

maximum iterations


**Постройте график зависимости ошибки от номера итерации**

In [142]:
import plotly.express as px

fig = px.line(
    x=range(len(err)), #ось абсцисс
    y=err, #ось ординат
    height=500, #высота
    width=1000, #ширина
    title='MSE (y) vs iter (x)' #заголовок
)
fig.show()
fig.write_html("MSE vs iter.html")

[Ссылка на график для git](https://drive.google.com/file/d/1CBaKoyKvS-1sJzDWd8KlU2HixG3c9R1F/view?usp=share_link)

**Выведите вектор весов, к которому сошелся метод.**

In [143]:
w_res

array([[14.0225    ],
       [55.39950779],
       [39.33467332],
       [-0.16794666]])

**Выведите среднеквадратичную ошибку на последней итерации.**

In [144]:
err[-1]

2.7842770681012223

*Дополнительно проверим себя встроенным регрессором*

In [145]:
from sklearn.linear_model import SGDRegressor
model = SGDRegressor(max_iter=10000,penalty = None, learning_rate = 'constant', eta0=0.3, random_state=42)
model.fit(X,y)
y_pred = model.predict(X)
print(mean_squared_error(y_pred, y))
model.coef_

2.8721585885332126


c:\Users\killr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



array([ 6.86428815e+00,  5.58415985e+01,  3.96479420e+01, -4.06974627e-02])

***Вывод***

* *Подбирать параметры для стахостического градиентного спуска ОЧЕНЬ важно*
* *Результат разный с библиотечной моделью на тех же параметрах из-за randomstate, я думаю*
* *Коэфиценты и результат не совпали, но похожи очень*